In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz, remez

In [ ]:
filter_length = 61

cutoff = 0.25
trans_band = 0.1
HB_pass = cutoff - (trans_band/2)
HB_stop = cutoff + (trans_band/2)

halfband_filter = remez(filter_length, [0, HB_pass, HB_stop, 0.5], [1, 0])

w, h = freqz(halfband_filter)

plt.figure(figsize=(10, 3))

plt.subplot(1, 2, 1)
plt.plot(halfband_filter)
plt.title("Impulse Response (sinc)")
plt.xlabel("samples (n)")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(w, 20*np.log10(np.maximum(np.abs(h), 1e-12)))
plt.title("Magnitude response")
plt.xlabel("Frequency (hz)")
plt.grid(True)
plt.show()

print(np.sum(halfband_filter))
print(np.sum(np.absolute(halfband_filter)))

In [ ]:
sound_files = []
for file in os.listdir():
    if file.endswith(('.bin')):
        sound_files.append(file)
print(sound_files)

In [ ]:
OVERSAMPLING = 64
fig, axs = plt.subplots(len(sound_files), 1, figsize=(50, 2 * len(sound_files)), sharex=True)
scale = np.iinfo(np.int16).max
num_halfband_passes = np.log2(OVERSAMPLING).astype(np.int8)

for i, file in enumerate(sound_files):
    raw_data = np.fromfile(file, dtype='uint8')
    bit_data = np.unpackbits(raw_data)
    data = bit_data.astype(np.float32) * 2 - 1
    for j in range(num_halfband_passes):
        data = np.convolve(data, halfband_filter)[::2]    
    audio = np.round(data * scale).astype(np.int16)
    output = 'pcm-' + file
    audio.tofile(output)
    simplified= data[::320]
    axs[i].plot(simplified)
    axs[i].set_title(file, fontsize=30)
    axs[i].tick_params(axis='both', which='major', labelsize=24)
    axs[i].set_ylim(-1, 1)
    axs[i].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    axs[i].grid(True)
plt.tight_layout()
plt.show()